# Ejen AI Azure dengan Sokongan Protokol Konteks Model (MCP) - Python

Notebook ini menunjukkan cara menggunakan Ejen AI Azure dengan alat Protokol Konteks Model (MCP) dalam Python. Ia menunjukkan cara untuk mencipta ejen pintar yang boleh memanfaatkan pelayan MCP luaran (seperti Microsoft Learn) untuk keupayaan yang dipertingkatkan menggunakan pengesahan tanpa kunci.


## Pasang Pakej Python yang Diperlukan

Pertama, kita perlu memasang pakej Python yang diperlukan:
- **azure-ai-projects**: SDK Projek Azure AI Teras
- **azure-ai-agents**: SDK Ejen Azure AI untuk mencipta dan mengurus ejen
- **azure-identity**: Menyediakan pengesahan tanpa kunci menggunakan DefaultAzureCredential
- **mcp**: Pelaksanaan Protokol Konteks Model untuk Python


## Kelebihan Pengesahan Tanpa Kunci

Notebook ini menunjukkan **pengesahan tanpa kunci** yang memberikan beberapa kelebihan:
- ✅ **Tiada kunci API untuk diuruskan** - Menggunakan pengesahan berasaskan identiti Azure
- ✅ **Keselamatan yang dipertingkatkan** - Tiada rahsia disimpan dalam kod atau fail konfigurasi
- ✅ **Putaran kelayakan automatik** - Azure menguruskan kitaran hayat kelayakan
- ✅ **Kawalan akses berasaskan peranan** - Menggunakan Azure RBAC untuk kebenaran yang lebih terperinci
- ✅ **Sokongan pelbagai persekitaran** - Berfungsi dengan lancar di persekitaran pembangunan dan pengeluaran

`DefaultAzureCredential` secara automatik memilih sumber kelayakan terbaik yang tersedia:
1. **Identiti Terurus** (apabila dijalankan di Azure)
2. Kelayakan **Azure CLI** (semasa pembangunan tempatan)
3. Kelayakan **Visual Studio**
4. **Pembolehubah persekitaran** (jika dikonfigurasi)
5. Pengesahan **penyemak imbas interaktif** (sebagai pilihan terakhir)


## Tetapan Pengesahan Tanpa Kunci

**Prasyarat untuk pengesahan tanpa kunci:**

### Untuk Pembangunan Tempatan:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Untuk Persekitaran Azure:
- Aktifkan **Identiti Terurus yang Ditetapkan oleh Sistem** pada sumber Azure anda
- Tetapkan **peranan RBAC** yang sesuai kepada identiti terurus:
  - `Cognitive Services OpenAI User` untuk akses Azure OpenAI
  - `AI Developer` untuk akses Projek AI Azure

### Pembolehubah Persekitaran (Pilihan):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Tiada kunci API atau rentetan sambungan diperlukan!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Import Perpustakaan Diperlukan

Import modul Python yang diperlukan:
- **os, time**: Perpustakaan standard Python untuk pembolehubah persekitaran dan kelewatan
- **AIProjectClient**: Klien utama untuk Projek AI Azure
- **DefaultAzureCredential**: Pengesahan tanpa kunci untuk perkhidmatan Azure
- **Kelas berkaitan MCP**: Untuk mencipta dan mengurus alat MCP serta mengendalikan kelulusan


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Konfigurasi Tetapan Pelayan MCP

Tetapkan konfigurasi pelayan MCP menggunakan pembolehubah persekitaran dengan nilai lalai sebagai sandaran:
- **MCP_SERVER_URL**: URL pelayan MCP (lalai kepada API Microsoft Learn)
- **MCP_SERVER_LABEL**: Label untuk mengenal pasti pelayan MCP (lalai kepada "mslearn")

Pendekatan ini membolehkan konfigurasi yang fleksibel merentasi pelbagai persekitaran.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Cipta Klien Projek Azure AI (Pengesahan Tanpa Kunci)

Inisialisasi klien Projek Azure AI menggunakan **pengesahan tanpa kunci**:
- **endpoint**: URL endpoint projek Azure AI Foundry
- **credential**: Menggunakan `DefaultAzureCredential()` untuk pengesahan yang selamat dan tanpa kunci
- **Tiada kunci API diperlukan**: Secara automatik mengesan dan menggunakan kelayakan terbaik yang tersedia

**Aliran Pengesahan:**
1. Memeriksa Managed Identity (dalam persekitaran Azure)
2. Beralih kepada kelayakan Azure CLI (untuk pembangunan tempatan)
3. Menggunakan sumber kelayakan lain yang tersedia jika diperlukan

Pendekatan ini menghapuskan keperluan untuk mengurus kunci API atau rentetan sambungan dalam kod anda.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Cipta Definisi Alat MCP

Cipta alat MCP yang berhubung dengan pelayan MCP Microsoft Learn:
- **server_label**: Pengenal untuk pelayan MCP
- **server_url**: URL endpoint pelayan MCP
- **allowed_tools**: Senarai pilihan untuk mengehadkan alat yang boleh digunakan (senarai kosong membenarkan semua alat)

Alat ini akan membolehkan ejen mengakses dokumentasi dan sumber Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Cipta Ejen dan Laksanakan Perbualan (Aliran Kerja Tanpa Kunci)

Bahagian komprehensif ini menunjukkan keseluruhan **aliran kerja ejen tanpa kunci**:

1. **Cipta Ejen AI**: Sediakan ejen dengan model GPT-4.1 nano dan alat MCP
2. **Cipta Thread**: Wujudkan thread perbualan untuk komunikasi
3. **Hantar Mesej**: Tanyakan ejen tentang perbezaan antara Azure OpenAI dan OpenAI
4. **Kendalikan Kelulusan Alat**: Luluskan panggilan alat MCP secara automatik apabila diperlukan
5. **Pantau Pelaksanaan**: Jejaki kemajuan ejen dan kendalikan sebarang tindakan yang diperlukan
6. **Paparkan Hasil**: Tunjukkan perbualan dan butiran penggunaan alat

**Ciri-ciri Tanpa Kunci:**
- ✅ **Tiada rahsia yang dikodkan secara keras** - Semua pengesahan dikendalikan oleh identiti Azure
- ✅ **Selamat secara lalai** - Menggunakan kawalan akses berasaskan peranan
- ✅ **Penyebaran dipermudahkan** - Tiada pengurusan kelayakan diperlukan
- ✅ **Mesra audit** - Semua akses dijejaki melalui identiti Azure

Ejen akan menggunakan alat MCP untuk mengakses sumber Microsoft Learn dengan keselamatan penuh dan tanpa pengurusan kunci API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
